### Logistic Regression Project in Spark:
    With titanic dataset

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression

In [2]:
spark = SparkSession.builder.appName('log_reg_project').getOrCreate()

In [3]:
data = spark.read.csv('../datasets/titanic.csv',inferSchema=True,header=True)

In [9]:
data.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [4]:
data.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

### Select the columns that we'll use in our model

In [10]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [5]:
cols = data.select('Pclass',
 'Survived',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked')

#### Drop any null values

In [6]:
final_data = cols.na.drop()

In [7]:
final_data.printSchema()

root
 |-- Pclass: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)



### Convert categorical column to numerical

In [8]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [9]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
# A B C
# 0 1 2
# ONE HOT ENCODER
# A = [1,0,0]
# B = [0,1,0] etc.
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [10]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

### Assemble all columns to make a feature column

In [11]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','Age','SibSp','Parch','Fare','EmbarkVec'],outputCol='features')

In [12]:
log_reg = LogisticRegression(featuresCol='features',labelCol='Survived')

### Creating the Pipeline

In [13]:
from pyspark.ml.pipeline import Pipeline

In [14]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,gender_encoder,
                            embark_encoder,assembler,log_reg])

In [15]:
# Train Test Split
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [16]:
model = pipeline.fit(train_data)

In [17]:
result = model.transform(test_data)

In [18]:
result.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [20]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

In [21]:
AUC = evaluator.evaluate(result)

In [22]:
AUC

0.8438128050531152

In [23]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [24]:
acc_eval = MulticlassClassificationEvaluator(labelCol='Survived',metricName='accuracy')
print(acc_eval.evaluate(result))

0.8476190476190476


#### We've got 84 % accuracy in this model